In [7]:
import pandas as pd
import cv2
import os, sys
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline
print "OpenCV Version : %s " % cv2.__version__

OpenCV Version : 2.4.9 


In [8]:
# Read image name
df = pd.read_csv('../data/data.csv')

df.head()

,target_market,score,cover_image,sugar_result,fiber_result,sodium_result,sugar_per_serving,fiber_per_serving,sodium_per_serving,calories_per_serving,serving_size,index,url
0,Family,50,http://cerealfacts.org/media/cereal_images/Ann...,24%,3%,293,7,1,85,120,29,1,http://cerealfacts.org/cereal_nutrition_scores...
1,Family,52,http://cerealfacts.org/media/cereal_images/Ann...,24%,3%,310,7,1,90,110,29,2,http://cerealfacts.org/cereal_nutrition_scores...
2,Family,50,http://cerealfacts.org/media/cereal_images/Ann...,24%,3%,293,7,1,85,120,29,3,http://cerealfacts.org/cereal_nutrition_scores...
3,Family,64,http://cerealfacts.org/media/cereal_images/Ann...,7%,3%,379,2,1,110,120,29,4,http://cerealfacts.org/cereal_nutrition_scores...
4,Family,58,http://cerealfacts.org/media/cereal_images/Bar...,23%,10%,267,7,3,80,120,30,5,http://cerealfacts.org/cereal_nutrition_scores...


In [9]:
def gen_file_name(row):
    file_name = row.cover_image.split('/')[len(row.cover_image.split('/'))-1]
    file_name = 'img_' + str(row['index']).zfill (3) + '_' + (file_name).lower()
    return(file_name)

df['file_name'] = df.apply(gen_file_name, axis=1)

In [10]:
df.head()

,target_market,score,cover_image,sugar_result,fiber_result,sodium_result,sugar_per_serving,fiber_per_serving,sodium_per_serving,calories_per_serving,serving_size,index,url,file_name
0,Family,50,http://cerealfacts.org/media/cereal_images/Ann...,24%,3%,293,7,1,85,120,29,1,http://cerealfacts.org/cereal_nutrition_scores...,img_001_anniesfruitybunnies.jpg
1,Family,52,http://cerealfacts.org/media/cereal_images/Ann...,24%,3%,310,7,1,90,110,29,2,http://cerealfacts.org/cereal_nutrition_scores...,img_002_annieshoneybunnies.jpg
2,Family,50,http://cerealfacts.org/media/cereal_images/Ann...,24%,3%,293,7,1,85,120,29,3,http://cerealfacts.org/cereal_nutrition_scores...,img_003_anniescinnamonrollbunnyos.jpg
3,Family,64,http://cerealfacts.org/media/cereal_images/Ann...,7%,3%,379,2,1,110,120,29,4,http://cerealfacts.org/cereal_nutrition_scores...,img_004_anniesorganicbunnyos.jpg
4,Family,58,http://cerealfacts.org/media/cereal_images/Bar...,23%,10%,267,7,3,80,120,30,5,http://cerealfacts.org/cereal_nutrition_scores...,img_005_barbarasbakerypuffinspuffscrunchycocoa...


In [11]:
sample_img = df['file_name']
print type(sample_img[0])
print sample_img.head()
print len(sample_img)

<type 'str'>
0                      img_001_anniesfruitybunnies.jpg
1                       img_002_annieshoneybunnies.jpg
2                img_003_anniescinnamonrollbunnyos.jpg
3                     img_004_anniesorganicbunnyos.jpg
4    img_005_barbarasbakerypuffinspuffscrunchycocoa...
Name: file_name, dtype: object
260


In [12]:
# Find best contour (longest contour is the best contour)
def find_best_contour(contours):
    longest_index = 0
    longest_length = len(contours[0])
    for i in range(len(contours)):
        if len(contours[i]) > longest_length:
            longest_index = i
            longest_length = len(contours[i])
            
    return longest_index

In [14]:
# Remove background for first 10 images in kelvin_cropped_test

for i in range(10): 
    # Load the image
    img_name = '../img/' + sample_img[i]
    img = cv2.imread(img_name)
    gray = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
    
    ret,thresh = cv2.threshold(gray,250,255,cv2.THRESH_BINARY_INV) # Tune the threshold here
    
    # Find and draw the contour
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1,(0,0,255),3)
    
    # Convert jpg to png
    img_jpg = Image.open(img_name)
    img_png = img_jpg.convert("RGBA")
    png_name = '../code/' + sample_img[i][:-4] + '.png'
    img_png.save(png_name, "PNG")
    
    alpha_img = cv2.imread(png_name,cv2.IMREAD_UNCHANGED)
    
    # Convert alpha values outside contour to be transparent (shadow removed)
    for y in range(alpha_img.shape[1]):
        for x in range(alpha_img.shape[0]):
            if cv2.pointPolygonTest(contours[find_best_contour(contours)], tuple([y, x]), False) == -1.0:
                alpha_img[x, y] = [255, 255, 255, 0]
    
    # Save the cropped file
    cropped_img_name = '../img/kelvin_cropped_test/' + sample_img[i][:-4] + '_cropped.png'
    cv2.imwrite(cropped_img_name, alpha_img)
    
    # Remove the png file
    os.remove(png_name)